In [1]:
# comment out to run on drive + colab 

# ! pip install transformers_interpret

In [2]:
# from google.colab import drive
# 
# drive.mount('/content/drive/')

In [3]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

In [4]:
# comment out the filepath to run on drive + colab 
filepath = "../models/scenario_level/anomic_wernicke_control"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/anomic_wernicke_control"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True, 
                                          truncation=True, return_tensors="pt", 
                                          add_special_tokens=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model.config

DistilBertConfig {
  "_name_or_path": "../models/anomic_wernicke_control/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "CONTROL",
    "1": "ANOMIC",
    "2": "WERNICKE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "ANOMIC": 1,
    "CONTROL": 0,
    "WERNICKE": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.39.0.dev0",
  "vocab_size": 30524
}

In [6]:
model.config.id2label

{0: 'CONTROL', 1: 'ANOMIC', 2: 'WERNICKE'}

In [7]:
def interpret(data):
    for text, label in zip(list(data["preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device) 
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits
                    
                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512) 
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:    
                with torch.no_grad():
                    logits = model(**inputs).logits
                    
                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [8]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios 
texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [9]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [10]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  yeah. [FP] when i when i nine eleven hit i was in my office. [FP] i wasn't the girls were out front watching the television set. and sulton came running in to tell me that something happened or i don't know. she just said it happened or something. and i went out there find out. and from that point on we just watched t v. [FP] [FP] i think [FP] firstnamea was there. and firstnames was there. [FP] who was it? they all came at different times. i mean she firstnames was there. i was there. [FP] firstnamed was there. firstnamedb was there. [FP] and eventually two others came in so there were all people. you know all we did was sit there because we watched where where it where each step. the first one it hit. and it started crumbling. and it just it then just completely crumbled. and like i said all we did was watch it crumble crumble and into nothing. we watched [FP] four four [FP] we watched all four buildings collapse. [FP] and like i said all we did was

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] talking [FP] it's good.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.17),CONTROL,-2.03,[CLS] [FP] talking [FP] it ' s good . [SEP]
n/a,(0.86),ANOMIC,1.69,[CLS] [FP] talking [FP] it ' s good . [SEP]
n/a,(0.55),WERNICKE,1.05,[CLS] [FP] talking [FP] it ' s good . [SEP]


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  could you talk could you speak?


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.38),CONTROL,-2.10,[CLS] could you talk could you speak ? [SEP]
n/a,(0.70),ANOMIC,2.49,[CLS] could you talk could you speak ? [SEP]
n/a,(0.41),WERNICKE,-2.04,[CLS] could you talk could you speak ? [SEP]


...

----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  okay. good? good.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.26),CONTROL,-2.32,[CLS] okay . good ? good . [SEP]
n/a,(0.82),ANOMIC,2.42,[CLS] okay . good ? good . [SEP]
n/a,(0.45),WERNICKE,-0.61,[CLS] okay . good ? good . [SEP]


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  [FP]. [FP] getting better. getting better. [FP] acupuncture. saturday each saturday. [FP] needles needles. yeah, yeah. because a vegetable. vegetable all the time. [FP] two years. [FP] one year couldn't talk and and then this and then. [FP] [FP] head opened up. and i'm in one months coma. and one year couldn't talk. [FP] but [FP] [FP] father mother pass away. old age. and [FP] but [FP] getting better and getting better. yeah. [FP] [FP] [FP] this and then acupuncture.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  [FP] it's just a wonderful thing to keep. i need more and more work. and everything that i see even like dying every day. i see more time getting better. a lot more. oh this anything that's open. i i'm seeing everything that's here. i don't see all these today. you open them. and i don't see all the papers that you stand. i need more more words. yes. yes yes.


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [11]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [12]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  oh. before my stroke. yeah. yeah it is. yeah. no this was [FP] when i big big kid big kid. and i was learning how to spell. i can't say that. oh. uhhuh. no i was a big person. yeah. a big person. and it was where i learned learned how to speak. oh [FP] that wasn't it. with my with my what what i did for a living. yeah. speech pathologist. yeah. uhhuh. sort of. uhhuh. yeah yeah. it was i i learned if it was in the working place. i can't i can't. yes yeah. uhhuh. yeah. well i'm not sure that it's speaking but. yeah uhhuh. yeah yeah. uhhuh. no. they were helping children but i didn't help them. i helped them tell the teachers i mean be the teachers. yeah. uhhuh. i worked for a school but but for the teachers and not for students. uhhuh. yeah. uhhuh. right. it helped the teachers learn what to do. yes. uhhuh. yeah.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  okay. okay. so i guess i would say [FP] [UP] [FP] my wedding. [FP] so i i well [FP] so i was [FP] an internship. [FP] i was in college. and i was doing an internship. and [FP] [FP] my husband was working. and [FP] and i actually went [FP] and asked [FP] [UP] my husband [FP] would i would he would like to go to lunch. yeah mhm. because i was [FP] i was one of those people that i was very bubbly bubbly. yeah. yeah. and so. yeah. and [FP] [FP] firstnamec was totally shy. so yeah. so i asked [FP] i asked it asked him to lunch. yeah. and [FP] and then we dated [FP] for five years. yeah. [FP] and and then we got marriage. yes. so [FP] it's been now it's been [FP] [UP] fourteen years. yeah. so it's it's. well yes absolutely. yes.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  it might have been the year before last. it was christmas time getting towards christmas time. and i was to meet some friends for lunch at zaytinia downtown. and i realized when i got down there that i was an hour early. it was kinda cold. dc sets up a [FP] a kind of a christmas bazaar down on g street? i guess it's g street. and they had closed the street to traffic. and there were wonderful little kiosks along the street with really nice stuff. and i was kinda cold so i bought a wool beret. and then i felt fine. i walked around. and i walked i was walking up seventh street. and up ahead of me on the right in descending letters it said regal cinema. just across the street was another sign also descending that said legal seafood. and on the window of the bank facing me right below that sign was coming soon eagle bank. i love i love washington.


----------------------------------------------------------------------
True:  CONTROL Pred:  ANOMIC
Text:  and i [FP] was one of the ones selected to be washed out. and i couldn't figure out why i was washed out in that i was as smart as [FP] you know i i could as the people around me. and my brother was in the class. and and i and i knew his capabilities. and [FP] but they really discovered that i couldn't pay attention. they had a it was an early [FP] thing of [FP] of concentration. it was a a disc on a like a [FP] record player. and the disc would go around. and you hadta hadta keep track hadta keep the electrode on top of that disc. and i know i didn't do well in that because i just i i wasn't a good concentrator. my brother was in the same class. and he was a good concentrator. i mean i remember him listening to the radio to jack armstrong. and you know the whole house would be falling down. and he he wouldn't come. but he wouldn't come to dinner when he was listening to the radio

----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  of what darling? yes. for me for me if i'm alone i'm okay. i remember one thing that's when i i decide to have a divorce. well [FP] i had two children. i'd i i had been [FP] during the year of the when it was the war. and [FP] and we lived together for i think it was probably twelve or hours maybe twelve or fourteen hours. we lived in another part of area way out in [FP] [FP] western near where you are. we were in greensburg. do you know where that is? that's where we that's where we lived. and [FP] and he had a he had [FP] [FP] a a father that he would drove out a great. i mean it was we had [FP] an old house. [FP] that was where my parents were still here. and i did that for a small period of time. and then and then he was very it was very not good there. yeah right. that was long time ago. and and mean not only i but i was still married for my two children. and and he was good

----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  it could be anything. i like. i like driving the car. [FP] i like going on driving. i i like anything. i bought i i i do anything. i [FP] will do anything. [FP] it's weird to say. i will do any. mhm. i got married. i got [FP] [FP] [FP] i heard a lot of things. it it's a quench. which one you want? oh ours was it's just like because our ours was just a separate ceremony. her. it's me and her. it's us. mhm.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  you want a good time? sad. oh to have my wife. have her. and it last [FP]. that's how long. and we were very happy. no no no no no no. it was four five seven eight eight. no to four feet. four. i could see her. bitter. only at timing. it's only short. yeah. two. no. one two three. yes. [FP] to be here. right rare. we get married. oh yeah. first one, long time. and then then i found the other one, took the new one. and i go with her. and it's her also lost. and then he came with the other. so that's why he came together. where [FP] we worked. yeah in the tear. in the air force.


----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [13]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [14]:
interpret(data)

True:  ANOMIC Pred:  CONTROL
Text:  and she had two children and he had one. and they all went to live together in his house. he passed away and she raised his daughter as her servant. and her children were always dressed pretty. and cinderella always hadta be [FP] the one that did everything. she cleaned up. she cooked. she made their dresses. [FP] alls i can think of is cinderella cinderella can we. the prince's father had [FP] like [FP] a servant. [FP] they sent out notes saying that they were gonna have a ball. and everybody was invited to the ball. and cinderella hadta fix their dresses [FP] the girls' dresses. and cinderella wanted to go to the ball too. so the the mice the mice went and made cinderella's dress out of her mother's dress. and then when the wicked stepmother saw it and the daughters they ripped it up. and cinderella was all upset. so she was crying. and her fairy godmother came. and she made her a dress and took some of the animals and made them into a some horses 

----------------------------------------------------------------------
True:  ANOMIC Pred:  CONTROL
Text:  there were two sisters and the mother. [FP] and she was a waitress in the house or working in the house or doing cleaning or something else like that. and then the girls were sposta go to a a a party. and but cinderella wasn't going to go to the party. [FP] and she [FP] called to her fairy godmother to ask about going to the party because she wanted to go also. and the fairy godmother said [FP]. i'll arrange for a dress for you. and she worked it out. and then [FP] cinderella went to the party. and [FP] at the party she learned to dance. [FP] and actually she danced with the prince. and but it was going to be twelve o'clock. and at the end of that she was going to [FP] she would hafta leave to go home before twelve o'clock. and the prince hadta hadta find the slipper which she left. and [FP] they tried to find her here there or the other place. and then the prince found the slippe

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] she went to the wedding. he's looking for [FP] [FP] just somebody to dance with. it's hard for me to. i i i i'm seeing in my in my head. oh god. they danced until twelve o'clock. and he she knows. i gotta get out of here. i gotta get going. see you later buddy! and she and she falls. i'm tired. i gotta go at twelve o'clock. i hafta go. and she and then the those girls made fun of her. it's he's he's he he he filled the shoe. he's got the shoe. and he goes. oh oh my look at this! oh. i'm sorry [FP].


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and they always gave her the dirty jobs to do around the house. they and these the these the wicked the the wicked the nasty stepsisters were invited to [FP] a ball in the castle. and they [FP] got dressed up to go. and cinderella was to stay at home and not go to the ball. the [FP] the good fairy came and [FP] made her waved her magic wand and gave cinderella a wonderful dress and and [FP] slippers [FP] glass slippers to wear to the ball. and then she went to the ball where she [FP] captured the interest of the prince who danced with her and and [FP] fell in love with her. and but the problem was the [FP] cinderella hadta leave at midnight. because the coach that she came in, which the good fairy had created from a pumpkin and mice, [FP] would change back into a pumpkin. and [FP] cinderella would be [FP] back in her old [FP] in her old clothes at that at midnight as well. so she

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  two little girls. and they are sposta live happily ever after. but the stepmother is not very nice to cinderella. makes her be the servant for these two girls. and as they grow up they don't get very nice. they're not nice at all to cinderella. she's relevated to [FP] not not comfortable quarters. and then the [FP] prince is gonna have a ball. and she everybody's all the eligible young ladies are invited because he's sposta [FP] pick a wife. so she's not allowed to go. she thinks she's sposta go too. but they won't let her. so they all go off to the ball. and she has a fairy godmother that comes and decks her all out in a coach and beautiful gown and etcetera. and she goes. and she dances. the only thing is she hasta be home by midnight. otherwise everything turns back into what it was before. she loses a slipper as she leaves. because she'd been dancing with the prince and they 

----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  she was [FP] angel for. she was [FP] for for someone else. the other children children for her are three children or whatever. with her it was very closed walking in. are going for the party. and she was fen people for for the present. the present [UP] was supposed to be [FP] thirty [FP] thirty or something. she [FP] had a ranned from home she huddled. the [FP] [UP] people were. they found her letter. and the the other people wed they found her. found her for the and the calls this one so.


----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [15]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [16]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  ten eleven twelve thirteen fourteen fifteen years ago. yeah. [FP] in in the club. [FP] and [FP] [FP] [FP] libertyville. [FP] i [FP] was exercising [FP] [FP] by myself in the room. and [FP] [FP] [FP] dancing [FP] [FP] [FP] ballroom dancing. and then i go to the [FP] locker room. and arm is [FP] [FP] [FP] [FP] [FP] [FP] [FP] [FP] [FP] frozen frozen. [FP] maybe the headache comes back. and [FP] and [FP] i get dressed and go to [FP] upstairs. and firstnamed says. what's wrong with me? what's wrong with him? what's wrong with me? no. what's wrong with [FP] him? and and then [FP] firstnamed and i go to the [FP] hospital right around the corner and had had [FP] the stroke. yeah. and and it's since then [FP] talking is problem. [FP] i say. oh canada i home and native land. and that's it. yeah! and [FP] [FP] [FP] [FP] [FP] [FP] [FP] [FP] toronto exercise my [FP] [FP] brain brain q. and i go [FP] [FP] [FP] [FP] [FP] [FP] possibilities for the [FP] [FP] writing 

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  and i just fell right on the floor. and then i fell into my into the cabinets. and and my the kids were coming into the house. that happened in the morning. and one of my dads is a state trooper. and he was able to right away. but i i i you know a parent called up the ambulance. i didn't particularly like the [FP] the hospital. i was there for nine days and i you know they were rough and gruff. and because i i hadta go to the bathroom all the time. i pee all the time. and so then you know they'd hafta put the thing under me. and they wouldn't let me walk and talk and do anything so i didn't like that part. but the rehab i really liked. i did. [FP] i was just telling somebody this the other day. i just heard that from [FP] firstnameb and firstnamem i think. and [FP] he he sat i started to cry. and i and he put he sat right on the floor. and he puts you know. and he said. you're gonn

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  half of my body is numb and [FP] diminished i can't feel. [FP] half. real [FP] suddenly. it snapped. and no speech at all. and [FP] twelve years ago [FP] i have a baby and nine months old. and [FP] [FP] the eyes it's weird because narrow very narrow [UP] [FP] eyes. it's i [FP] i can't firstnamem is no memory. firstnamem. now i say it but it's [FP] very odd. mhm and [FP] two months in the hospital. and [FP] multiple sclerosis. mhm yeah yeah. [FP] a lot of things. [FP] i volunteer everywhere. it's [FP] [FP] flower shuttle. [FP] volunteer at wake med firstnamec. [FP] [FP] ben's school. [FP] keep busy. but [FP] it's [FP] half an hour each day i rest. doesn't flat [FP] no reclining flat. [FP] [FP] watching t v it's it's [FP] charge it up.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  and i was driving from down on the beach. and i drove home. and i'd got in the my in my house then i went in my bed. and i knew there was something wrong with me. and i don't know why but i must have put the phone and called my either one of my [FP] girls. and they came immediately with their father of of the one [FP] tom who is the father of one. get those people out of there. i mean i was awake of that. i said. please. and finally they said. well if you if we let us take you to the hospital right away we'll put it we'll call them away. the [FP] you know what it is, the bang bang bang and the whole thing. i can't make it better. so immediately we got to the cemetary the hospital. and when i said. i i i wanna go home and stay by myself and leave. so finally i did but i i was you know uncertain the way they treated me. and [FP] and then i had [FP] [FP] quite a few doctors that up 

----------------------------------------------------------------------
True:  WERNICKE Pred:  CONTROL
Text:  sometime in in yeah [FP] around in there sometime. i woke up and i was gonna go with my sister and all with somebody. and i didn't feel too too hot. so i called her and told her to come and tell me i didn't feel good. and she told me to took me to her normal little little place and they didn't even look at me. they took me automatically to the store to the car or to the hospital. and i don't know what they. and that's where i know. i haven't. i haven't. not in my book! everybody says that i'm always getting better but i don't feel like i am. i can't stand to do this junk stair by myself. i've fallen three times in a row the very same day. i just don't wanna be around it that much. and i do it here because i hafta and when i get home i'm in my my chair.


----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [17]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [18]:
interpret(data)

True:  ANOMIC Pred:  CONTROL
Text:  the [FP] daughter is crying because her cot is up i tree. and the husband volunteers to claim the tree and rescue the cat. and the father was trapped by the tree. and the dog is panicking barking. [FP] and i don't know were what the fireman is. maybe their mother called the firemen to rescue the husband maybe. and the fireman running in the ladder to rescue the husband. i i suspect the husband is rescued. and the cat is rescued too. the end.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  so she called for this man. and he he went out. and he couldn't get it. and the dog was barking. and so [FP] well he he [FP] dropped [FP] a ladder. so they called the [FP] [FP] firefighter. and they came over to get the cat and the man out of the chair tree. and [FP] then they went back to the place.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] that dog and a cat that ran [FP] up the tree, the cat. and the [FP] rider his [FP] the woman no little girl had a tricycle. and she's up in [FP] [FP] [UP] [FP] the little [FP] girl, she on the cat. and he he said. come here kitty, kitty. and [FP] his dad [FP] woman [FP] woman had a ladder. and the dog kick or the [FP] tree. and [FP] so the adder. [FP] no hmhm. so the firemen out on the tree. and this one he [FP] save him up the tree. and the bird is singing.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and [FP] the little girl probably went off got her dad or her neighbor. who brought his ladder over and [FP] laid it up on the tree. and [FP] [UP] in the interim the ladder got out from under him. and he's stuck up the tree. so now the dog is upset probably because he's not up the tree. and [FP] the cat seems comfortable. dad or the neighbor looks forlorn. and the fire department has just gotten gotten there. and are proceeding to come on the scene and hopefully get both out of the tree. and calm the dog down.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  and her mad pussycat went up the tree. and he wants to get her back. meanwhile one dog has come to catch him. so and and he don't doesn't want the dog go up. and he pushed the ladder outside. and somebody getting the ladder. oh.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  oh this is a story about a tree. [FP] there's a cat that's stuck in the tree [FP] probably because [FP] the dog has chased the cat into the tree. a little girl's crying because her cat is [FP] stuck in the tree. and she's come over on her tricycle. [FP] a man, probably the girl's father, has climbed the tree with a ladder. and he seems to be stuck too. and [FP] the fire department shows up with a bigger ladder. and presumably they're going to get the cat and the father out of the tree.


----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  mess i don't. my goodness. he doesn't want neither one. well he lets he has some on. and she said. i want him out but i don't get it. i don't get it there. she goes i hafta go. he can't go up that to get him. and he can't he can't hold to get it. so he he got. look at the. oh the. what is it? this one. and this whether he said. we have no and we that and we that. oh look at this at. i don't know what's he doing. what this? i don't know what that doing. i dunno.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  and the firemen the firemen firemen. an he's good get toes. so he's in [FP] in. so [FP] i don't know. he had.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  okay. the cat. and he's she says. and he and he said. the rope. and [FP] cat and [FP] now let's see the [FP] dog. and she says. and there's her pop. say. i can't get get rope. and [FP] there's a [FP] let's see bird.


----------------------------------------------------------------------
